# 4.5 Calibrating a model

In this tutorial, you will learn how to calibrate an agent-based model.

## Problem 1

We will calibrate the SIR model (beta and gamma) to some simple outbreak data.

## Solution 1

## Problem 2

Using a more realistic example, we will calibrate Starsim to some Kenya HIV data.

## Solution 2